In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import stats 

from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family']='Microsoft YaHei' 

### 0、WOE和IV计算

In [2]:
# 计算WOE和IV

def cal_woe(Y,X,bins):
    """ 计算卡方值
    Args:
        Y: 标签值
        X：变量值
        bins: 变量X的分箱
    Returns:
        result,woe,iv
    """
    
    total_bad = Y.sum()    # 总的坏样本数
    total_good = Y.count() - total_bad   # 总的好样本数
    
    df = pd.DataFrame({'X':X, 'Y':Y, 'bins':pd.cut(X,bins)})

    #分组计算好坏样本数
    bad=df.query("Y==1").groupby('bins')['Y'].count().reset_index(name='badnum')
    good=df.query("Y==0").groupby('bins')['Y'].count().reset_index(name='goodnum')

    result=pd.merge(bad,good,on='bins')


    result['badrate'] = result['badnum'] / total_bad
    result['goodrate'] = result['goodnum'] / total_good
    result['woe'] = np.log(result['goodrate'] / result['badrate'])
    result['iv']=(result['goodrate'] - result['badrate']) * result['woe']
    iv = result['iv'].sum()

    # woe值列表
    woe = list(result['woe'])
    
#     print("变量{0}的iv值为：{1}".format(X.name,round(iv,2)))
    
    plt.figure(figsize=(8,6))
    plt.bar(range(result.shape[0]),result['woe'],tick_label=result['bins'],label='WOE')
    plt.title("{0}(iv={1})".format(X.name,round(iv,2)))
    plt.xticks(rotation=30)
    plt.rc('legend', fontsize=10)
    plt.legend(loc='best')
    plt.show()
    
    return result,woe,iv

### 1、卡方分箱

基于卡方检验的连续变量最优分箱，实现步骤如下：  
* （1）给定连续变量 V，对V中的值进行排序，然后每个元素值单独一组，完成初始化阶段；  
* （2）对相邻的组，两两计算卡方值；  
* （3）合并卡方值最小的两组；  
* （4）递归迭代步骤2-3，直到满足停止条件。（一般是卡方值都高于设定的阈值，或者达到最大分组数等等）

In [4]:
def cal_chi(freq):
    """ 计算卡方值
    Args:
        freq: Array，待计算卡方值的二维数组，频数统计结果
    Returns:
        卡方值，float
    """
    
    # 计算每列的频数之和
    col_nums = freq.sum(axis=0)
    # 计算每行的频数之和
    row_nums = freq.sum(axis=1)
    # 计算总频数
    nums = freq.sum()
    # 计算期望频数
    E_nums = np.ones(freq.shape) * col_nums / nums
    E_nums = (E_nums.T * row_nums).T
    # 计算卡方值
    tmp_v = (freq - E_nums)**2 / E_nums
    # 如果期望频数为0，则计算结果记为0
    tmp_v[E_nums==0] = 0
    chi_v = tmp_v.sum()
    return chi_v


def chimerge_cut(data, var, target, max_group=None, chi_threshold=None):
    """ 计算卡方分箱的最优分箱点
    Args:
        data: DataFrame，待计算卡方分箱最优切分点列表的数据集
        var: 待计算的连续型变量名称
        target: 待计算的目标列Y的名称
        max_group: 最大的分箱数量（因为卡方分箱实际上是合并箱体的过程，需要限制下最大可以保留的分箱数量）
        chi_threshold: 卡方阈值，如果没有指定max_group，我们默认选择类别数量-1，置信度95%来设置阈值
        
    Returns:
        最优切分点列表，List
    """
    freq_df = pd.crosstab(index=data[var], columns=data[target])
    # 转化为二维数组
    freq_array = freq_df.values
    
    # 初始化箱体，每个元素单独一组
    best_bincut = freq_df.index.values
    
    # 初始化阈值 chi_threshold，如果没有指定 chi_threshold，则默认选择target数量-1，置信度95%来设置阈值
    if max_group is None:
        if chi_threshold is None:
            chi_threshold = chi2.isf(0.05, df = freq_array.shape[-1])
    
    # 开始迭代
    while True:
        min_chi = None
        min_idx = None
        for i in range(len(freq_array) - 1):
            # 两两计算相邻两组的卡方值，得到最小卡方值的两组
            v = cal_chi(freq_array[i: i+2])
            if min_chi is None or min_chi > v:
                min_chi = v
                min_idx = i
        
        # 是否继续迭代条件判断
        # 条件1：当前箱体数仍大于 最大分箱数量阈值
        # 条件2：当前最小卡方值仍小于制定卡方阈值
        if (max_group is not None and max_group < len(freq_array)) or (chi_threshold is not None and min_chi < chi_threshold):
            tmp = freq_array[min_idx] + freq_array[min_idx+1]
            freq_array[min_idx] = tmp
            freq_array = np.delete(freq_array, min_idx+1, 0)
            best_bincut = np.delete(best_bincut, min_idx+1, 0)
        else:
            break
    
    # 把切分点补上头尾
    best_bincut = best_bincut.tolist()
    best_bincut.append(data[var].min())
    best_bincut.append(data[var].max())
    best_bincut_set = set(best_bincut)
    best_bincut = list(best_bincut_set)
    
    best_bincut.remove(data[var].min())
    best_bincut.append(-np.inf)
    best_bincut.remove(data[var].max())
    best_bincut.append(np.inf)
    # 排序切分点
    best_bincut.sort()
    
    return best_bincut

In [5]:
# for i in range(len(freq)-1):
#     print(freq.index.values[i:i+2],cal_chi(freq.values[i:i+2]))
    
    

# for i in range(len(freq) - 1):
#     # 两两计算相邻两组的卡方值，得到最小卡方值的两组
#     v = cal_chi(freq.values[i:i+2])
#     if min_chi is None or min_chi > v:
#         min_chi = v
#         idx=i
#         min_idx = freq.index.values[i:i+2]

### 2、决策树分箱

基于CART算法的连续变量最优分箱，实现步骤如下：
* （1）给定连续变量 V，对V中的值进行排序；
* （2）依次计算相邻元素间中位数作为二值划分点的基尼指数；
* （3）选择最优（划分后基尼指数下降最大）的划分点作为本次迭代的划分点；
* （4）递归迭代步骤2-3，直到满足停止条件。（一般是以划分后的样本量作为停止条件，比如叶子节点的样本量>=总样本量的10%）

In [6]:
def dt_bins(X,Y,max_leaf_num): 
    """利用决策树获得最优分箱的边界值"""  
    
    """
    :param X: 待分箱特征
    :param Y: 目标变量
    :param max_leaf_num: 分箱数
    
    :return: 统计值、分箱边界值列表、woe值、iv值
    """
    bins = []  
    x = X.values  
    y = Y.values 
    clf = DecisionTreeClassifier(criterion='entropy',  # 信息熵最小化准则划分 
                                 max_leaf_nodes=max_leaf_num,  # 最大叶子节点数  
                                 min_samples_leaf = 0.05)  # 叶子节点样本数量最小占比 
    clf.fit(x.reshape(-1,1),y)  # 训练决策树 
     
    n_nodes = clf.tree_.node_count  
    children_left = clf.tree_.children_left  
    children_right = clf.tree_.children_right  
    threshold = clf.tree_.threshold  
     
    for i in range(n_nodes): 
        if children_left[i] != children_right[i] : # 获的决策时节点上的划分边界 
            bins.append(threshold[i]) 
    bins.sort() 
    min_x = -np.inf
    max_x = np.inf # 加0.1是为了考虑后续groupby操作时, 能包含特征最大值得样本 
    bins=[min_x]+bins
    bins = bins +[max_x]
    
    #计算woe和iv
    result,woe,iv=cal_woe(Y,X,bins)
    
    return result,bins,woe,iv

###  3、Best-KS分箱

基于最优KS的连续变量最优分箱，实现步骤如下：
* （1）给定连续变量 V，对V中的值进行排序；
* （2）每一个元素值就是一个计算点；
* （3）计算出KS最大的那个元素，作为最优划分点，将变量划分成两部分D1和D2；
* （4）递归迭代步骤3，计算由步骤3中产生的数据集D1 D2的划分点，直到满足停止条件。（一般是分箱数量达到某个阈值，或者是KS值小于某个阈值）


In [7]:
def bestks_bins(X,Y,n):
    """
    :param X: 待分箱特征
    :param Y: 目标变量
    :param max_leaf_num: 分箱数
    
    :return: 统计值、分箱边界值列表、woe值、iv值
    """
    total_bad = Y.sum()    # 总的坏样本数
    total_good = Y.count() - total_bad   # 总的好样本数
    
    bins=[]
    
    list1=X.unique()
    list1.sort()
    
    df=pd.DataFrame({"X":X,"Y":Y,"bins":pd.cut(X,list1)})
    df['bad']=df["Y"]
    df['good']=df['Y'].apply(lambda x:1 if x==0 else 0)

    
    bad=pd.DataFrame(df.groupby(['X'])["bad"].sum()).reset_index()
    good=pd.DataFrame(df.groupby(['X'])["good"].sum()).reset_index()
    
    bad['bad']=bad['bad'].cumsum()
    good['good']=good['good'].cumsum()
    
    df2=pd.merge(bad,good,on="X",how="left")
    
    
    df2['bad_pct']=df2['bad']/total_bad
    df2['good_pct']=df2['good']/total_good
    df2['ks']=abs(df2['good_pct']-df2['bad_pct'])
    
    #取ks最大的点作为切分点
    id_max=df2['ks'].idxmax()
    x_max=df2['X'][id_max]
    
    bins.append(x_max)
    
    bins.append(list1.min())
    bins.append(list1.max())
    bins.sort()
    return bins

### 4、其他分箱

In [8]:
def spearman_bins(Y, X, n):
    """
    :param Y: 目标变量
    :param X: 待分箱特征
    :param n: 分箱数初始值
    :return: 统计值、分箱边界值列表、woe值、iv值
    """
    r = 0    # 初始值
    total_bad = Y.sum()    # 总的坏样本数
    total_good = Y.count() - total_bad    # 总的好样本数
    
    # 分箱过程
    while np.abs(r) < 1:    
        df1 = pd.DataFrame({'X':X, 'Y':Y, 'bin':pd.qcut(X, n, duplicates='drop')})    # qcut():基于量化的离散化函数
        df2 = df1.groupby('bin')
        r, p = stats.spearmanr(df2.mean().X, df2.mean().Y)
        n = n - 1
        
        if n==1:
            break
        else:
            continue
    bins = []
    bins.append(-np.inf)
    for i in range(1, n+1):
        qua = X.quantile(i / (n+1))
        bins.append(round(qua, 6))
    bins.append(np.inf)
    bins =list(np.unique(bins))
    
    
    #计算woe和iv
    result,woe,iv=cal_woe(Y,X,bins)
    
    return result,bins,woe,iv